# Trabalho 1 - Machine Learning II 
Prof: Carlos Padilha

#### Alunos:  

Roberto A. Coutinho  
Thais Galho


## Sistemas com Multi-classificadores ou Ensembles

#### Este trabalho visa avaliar o entendimento em relaçãao á construção de sistemas com multi-classificadores ou ensembles. Para tal, os alunos deverão fazer o seguinte:


* Implementar o algoritmo AdaBoost (nos mesmos moldes que fizemos com o algoritmo Bagging).
    – Podem escolher qualquer tipo de classificador (MLP, SVM, etc).
* Processar os dados presente no arquivo sonar.all-data.
* Realizar treinamento e teste usando validação cruzada com 10 folds.
* Avaliar os resultados em termos de acurácia, recall e precisão.

Obs: O trabalho pode ser feito em dupla e deve ser enviado por email (carlos.engcomp@gmail.com).

In [1]:
import numpy as np
import pandas as pd

# Modelos
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# K-fold CrossValidation
from sklearn.model_selection import KFold

# K-fold CrossValidation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# Métricas
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [2]:
imported_data = pd.read_csv('sonar.all-data.csv', header=None)
imported_data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
# Separação entre dados e labels

labels = imported_data.iloc[:,-1]
data = imported_data.iloc[:,:-1]
len(data), len(labels)

(208, 208)

In [148]:
from sklearn.metrics import confusion_matrix

def getAccuracy(testset, predictions):
    correct = 0
    for id_test, test in enumerate(testset):
        if test == predictions[id_test]:
            correct += 1
    return (correct / float(len(testset))) * 100.0

def printCM(Y_test, predictions):
    cm = confusion_matrix(Y_test, predictions)
    print ('Confusion Matrix : ')
    print (cm)
    print
    tn = float(cm[0][0])
    fp = float(cm[0][1])
    fn = float(cm[1][0])
    tp = float(cm[1][1])

    actual_yes = fn+tp
    actual_no = tn+fp
    predicted_yes = fp+tp
    predicted_no = tn+fn

    total = float(len(imported_data))
    print ('Total : '+ str(total))

    acc = str(getAccuracy(Y_test, predictions))
    print ('Acurácia : ' + acc)

    misclassification_rate = round((fp+fn)/total,3) # Overall, how often is it wrong?
    print ('Misclassification rate : ' +str(misclassification_rate))

    true_positive = round(tp/actual_yes,3) # When it's actually yes, how often does it predict yes?
    print ('True positives : ' +str(true_positive))

    false_positive = round(fp/actual_no,3) # When it's actually no, how often does it predict yes?
    print ('False positives : ' +str(false_positive))

    specificity = round(tn/actual_no,3) # When it's actually no, how often does it predict no?
    print ('Specificity : ' +str(specificity))

    precision = round(tp/predicted_yes,3) # When it predicts yes, how often is it correct?
    print ('Precision : ' +str(precision))

    prevalence = round(actual_yes/total,3) # How often does the yes condition actually occur in our sample?
    print ('Prevalence : ' +str(prevalence))

    #f1 = round(2 * ((precision * true_positive) / (precision + true_positive)),3)
    #print ('F1 Score : ' +str(f1))
    
    return acc, precision

<h2>Treinamento</h2>

In [4]:

# utiliza 25% do dataset para teste
trainData, testData, trainLabels, testLabels = train_test_split(data, labels, 
                                                    train_size=0.75, 
                                                    test_size=0.25, 
                                                    stratify=labels)

print(len(trainData), len(trainLabels))
print(len(testData), len(testLabels))

(156, 156)
(52, 52)


In [158]:
scores = []

print("Inicializando vetor de pesos....")

# Initialize weights
w = np.ones(n_train) / n_train
print(w)

n_train, n_test = len(trainData), len(testData)
#pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]
    
# Fit um classificador
clf_tree = DecisionTreeClassifier(max_depth = 1, random_state = 1)

# Define quantos folds
kf = KFold(n_splits=10)
alpha = 0.01

print
print("Iniciando treinamento com 10 K-folds" )
print
    
kfold = 0    
for train_index, test_index in kf.split(trainData):
    print("################################################")
    print("K-fold : "+str(kfold+1))    
    print("################################################")
    #print(train_index, test_index)
    print
    
    # Obten os subdados de treinamento e teste no n fold
    #---------------------------------------------------------------------
    X_train, X_test = trainData.iloc[train_index,:], trainData.iloc[test_index,:]
    #print(len(X_train), len(X_test))

    y_train, y_test = trainLabels.iloc[train_index], trainLabels.iloc[test_index]
    #print(len(y_train), len(y_test))

    # Obten os sub-pesos de treinamento e teste no n fold
    #---------------------------------------------------------------------
    trainWeights = w[train_index]
    testWeights = w[test_index]
    
    # Treina o modelo de classificação
    #---------------------------------------------------------------------
    print("Treinando o modelo....")
    
    # Treina o classificador com os pesos de treinamento
    clf_tree.fit(X_train, y_train, sample_weight=trainWeights)

    # Classifica o treino
    pred_train_i = clf_tree.predict(X_train)
    #print(pred_train_i)

    # Classifica o teste
    pred_test_i = clf_tree.predict(X_test)
    #print(pred_test_i)        

    kfold += 1 
    print

    
    print
    print("...:::: Avaliação ::::....  ")
    print
    
    # Obtem o index dos erros da classificação de treino e teste
    #---------------------------------------------------------------------
    miss = [int(x) for x in (pred_train_i != y_train)]
    #print("Training Miss : "+str(miss))
    missTest = [int(x) for x in (pred_test_i != y_test)]
    #print("Testing Miss : "+str(missTest))

    
    # Equivale os valores entre 1/-1 para atualização dos pesos
    #---------------------------------------------------------------------
    miss2 = [x if x==1 else -1 for x in miss]
    #print("Training Miss2 : "+str(miss2))
    miss2Test = [x if x==1 else -1 for x in missTest]
    #print("Testing Miss2 : "+str(miss2Test))
    
    # Calcula o erro
    #---------------------------------------------------------------------
    err_m = np.dot(trainWeights,miss) / sum(trainWeights)
    print("Error : "+str(err_m))

    # Calcula o Alpha 
    #---------------------------------------------------------------------
    alpha_m = alpha * np.log( (1 - err_m) / float(err_m))
    print("Alpha : "+str(alpha_m))

    # Mostra a Matriz de Confusão para treino e teste
    #---------------------------------------------------------------------
    print
    print(":: Treinamento :: ")
    print
    train_acc_score, train_precision_score = printCM(y_train, pred_train_i)
    
    print
    print(":: Teste ::")
    print
    test_acc_score, test_precision_score = printCM(y_test, pred_test_i)
    print
    
    
    # Atualiza os valores dos pesos
    #---------------------------------------------------------------------
    w = np.multiply(w, np.exp([float(x) * alpha_m for x in np.concatenate((miss2, miss2Test), axis=0)]))
    print("New weights : ")
    print(w)
    
    
    scores.append([kfold, train_acc_score, train_precision_score])

    print

Inicializando vetor de pesos....
[0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026 0.00641026
 0.00641026 0.00641026 0.0064

In [155]:
display(scores)

[[1, '77.8571428571', 0.797],
 [2, '78.5714285714', 0.806],
 [3, '77.8571428571', 0.776],
 [4, '77.1428571429', 0.768],
 [5, '77.1428571429', 0.759],
 [6, '77.8571428571', 0.785],
 [7, '78.7234042553', 0.817],
 [8, '80.1418439716', 0.797],
 [9, '78.7234042553', 0.787],
 [10, '78.0141843972', 0.817]]

In [154]:
print(scores)

[[1, '77.8571428571', 0.797], [2, '78.5714285714', 0.806], [3, '77.8571428571', 0.776], [4, '77.1428571429', 0.768], [5, '77.1428571429', 0.759], [6, '77.8571428571', 0.785], [7, '78.7234042553', 0.817], [8, '80.1418439716', 0.797], [9, '78.7234042553', 0.787], [10, '78.0141843972', 0.817]]
